<a href="https://colab.research.google.com/github/wanng-ide/Fengshenbang-colab/blob/main/finetune_taiyi_stable_diffusion_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🖌️ **Finetuning Taiyi-Stable-Diffusion Colab Example**

#####based on https://huggingface.co/IDEA-CCNL/Taiyi-Stable-Diffusion-1B-Chinese-EN-v0.1


# Installing fengshen framework

In [1]:
from IPython.display import clear_output

!pip install pytorch_lightning
!pip install transformers
!pip install deepspeed
!pip install diffusers
!pip install datasets
!pip install accelerate

!git clone https://github.com/IDEA-CCNL/Fengshenbang-LM

clear_output()
print("Done!")

Done!


In [2]:
import os

os.chdir('/content/Fengshenbang-LM')
print(os.getcwd())

/content/Fengshenbang-LM


# Building modules

In [3]:
import os
import torch
import argparse
from pytorch_lightning import (
    LightningModule,
    Trainer,
)
from pytorch_lightning.callbacks import (
    LearningRateMonitor,
)
from fengshen.data.universal_datamodule import UniversalDataModule
from fengshen.models.model_utils import (
    add_module_args,
    configure_optimizers,
    get_total_steps,
)
from fengshen.utils.universal_checkpoint import UniversalCheckpoint
from transformers import BertTokenizer, BertModel
from diffusers import AutoencoderKL, DDPMScheduler, StableDiffusionPipeline, UNet2DConditionModel
from torch.nn import functional as F
from fengshen.data.taiyi_stable_diffusion_datasets.taiyi_datasets import add_data_args, load_data


class StableDiffusion(LightningModule):
    @staticmethod
    def add_module_specific_args(parent_parser):
        parser = parent_parser.add_argument_group('Taiyi Stable Diffusion Module')
        parser.add_argument('--train_whole_model', action='store_true', default=False)
        return parent_parser

    def __init__(self, args):
        super().__init__()
        self.tokenizer = BertTokenizer.from_pretrained(
            args.model_path, subfolder="tokenizer")
        self.text_encoder = BertModel.from_pretrained(
            args.model_path, subfolder="text_encoder")  # load from taiyi_finetune-v0
        self.vae = AutoencoderKL.from_pretrained(
            args.model_path, subfolder="vae")
        self.unet = UNet2DConditionModel.from_pretrained(
            args.model_path, subfolder="unet")

        self.noise_scheduler = DDPMScheduler(
            beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000
        )
        self.save_hyperparameters(args)

    def setup(self, stage) -> None:
        if stage == 'fit':
            self.total_steps = get_total_steps(self.trainer, self.hparams)
            print('Total steps: {}' .format(self.total_steps))

    def configure_optimizers(self):
        model_params = [{'params': self.text_encoder.parameters()}]
        if self.hparams.train_whole_model:
            model_params.append({'params': self.unet.parameters()})
        return configure_optimizers(self, model_params=model_params)

    def training_step(self, batch, batch_idx):
        self.text_encoder.train()

        with torch.no_grad():
            latents = self.vae.encode(batch["pixel_values"]).latent_dist.sample()
            latents = latents * 0.18215

        # Sample noise that we'll add to the latents
        noise = torch.randn(latents.shape).to(latents.device)
        noise = noise.to(dtype=self.unet.dtype)
        bsz = latents.shape[0]
        # Sample a random timestep for each image
        timesteps = torch.randint(
            0, self.noise_scheduler.config.num_train_timesteps, (bsz,), device=latents.device)
        timesteps = timesteps.long()
        # Add noise to the latents according to the noise magnitude at each timestep
        # (this is the forward diffusion process)

        noisy_latents = self.noise_scheduler.add_noise(latents, noise, timesteps)
        noisy_latents = noisy_latents.to(dtype=self.unet.dtype)

        # Get the text embedding for conditioning
        # with torch.no_grad():
        encoder_hidden_states = self.text_encoder(batch["input_ids"])[0]

        # Predict the noise residual
        noise_pred = self.unet(noisy_latents, timesteps, encoder_hidden_states).sample

        loss = F.mse_loss(noise_pred, noise, reduction="none").mean([1, 2, 3]).mean()
        self.log("train_loss", loss.item(),  on_epoch=False, prog_bar=True, logger=True)

        if self.trainer.global_rank == 0 and self.global_step == 100:
            # 打印显存占用
            from fengshen.utils.utils import report_memory
            report_memory('stable diffusion')

        if self.trainer.global_rank == 0:
            if (self.global_step+1) % 5000 == 0:
                print('saving model...')
                pipeline = StableDiffusionPipeline.from_pretrained(
                    args.model_path, text_encoder=self.text_encoder, tokenizer=self.tokenizer,
                )
                self.trainer.current_epoch
                pipeline.save_pretrained(os.path.join(
                    args.default_root_dir, f'hf_out_{self.trainer.current_epoch}'))

        return {"loss": loss}

    def on_train_epoch_end(self):
        if self.trainer.global_rank == 0:
            print('saving model...')
            pipeline = StableDiffusionPipeline.from_pretrained(
                args.model_path, text_encoder=self.text_encoder, tokenizer=self.tokenizer,
            )
            self.trainer.current_epoch
            pipeline.save_pretrained(os.path.join(
                args.default_root_dir, f'hf_out_{self.trainer.current_epoch}'))

    def on_load_checkpoint(self, checkpoint) -> None:
        # 兼容低版本lightning，低版本lightning从ckpt起来时steps数会被重置为0
        global_step_offset = checkpoint["global_step"]
        if 'global_samples' in checkpoint:
            self.consumed_samples = checkpoint['global_samples']
        self.trainer.fit_loop.epoch_loop._batches_that_stepped = global_step_offset

# Settings

In [4]:
from pprint import pprint

args_parser = argparse.ArgumentParser()
args_parser = add_module_args(args_parser)
args_parser = add_data_args(args_parser)
args_parser = UniversalDataModule.add_data_specific_args(args_parser)
args_parser = Trainer.add_argparse_args(args_parser)
args_parser = StableDiffusion.add_module_specific_args(args_parser)
args_parser = UniversalCheckpoint.add_argparse_args(args_parser)

your_dataset_path = '/content/Fengshenbang-LM/fengshen/examples/finetune_taiyi_stable_diffusion/demo_dataset' #@param {type:"string"}
your_model_path =  'IDEA-CCNL/Taiyi-Stable-Diffusion-1B-Chinese-v0.1' #@param {type:"string"}
train_batch_size = '1' #@param {type:"string"}

message = [
    '--datasets_path', your_dataset_path,
    '--datasets_type', 'txt',
    '--model_path', your_model_path,
    '--train_batchsize', train_batch_size
]

args = args_parser.parse_args(args=message)

pprint(vars(args), width = 230)

{'accelerator': None,
 'accumulate_grad_batches': None,
 'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_epsilon': 1e-08,
 'amp_backend': 'native',
 'amp_level': None,
 'auto_lr_find': False,
 'auto_scale_batch_size': False,
 'auto_select_gpus': False,
 'benchmark': None,
 'center_crop': False,
 'check_val_every_n_epoch': 1,
 'dataloader_workers': 2,
 'datasets_name': None,
 'datasets_path': ['/content/Fengshenbang-LM/fengshen/examples/finetune_taiyi_stable_diffusion/demo_dataset'],
 'datasets_type': ['txt'],
 'default_root_dir': None,
 'detect_anomaly': False,
 'deterministic': None,
 'devices': None,
 'enable_checkpointing': True,
 'enable_model_summary': True,
 'enable_progress_bar': True,
 'every_n_epochs': None,
 'every_n_train_steps': None,
 'fast_dev_run': False,
 'filename': 'model-{epoch:02d}-{train_loss:.4f}',
 'gpus': None,
 'gradient_clip_algorithm': None,
 'gradient_clip_val': None,
 'inference_mode': True,
 'ipus': None,
 'learning_rate': 5e-05,
 'limit_predict_batches': 

# Start training

In [5]:
!nvidia-smi

Fri Nov 25 13:04:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:

model = StableDiffusion(args)
tokenizer = model.tokenizer
datasets = load_data(args, tokenizer=tokenizer)

def collate_fn(examples):
    # print(examples)
    input_ids = [example["instance_prompt_ids"] for example in examples]
    pixel_values = [example["instance_images"] for example in examples]

    pixel_values = torch.stack(pixel_values)
    pixel_values = pixel_values.to(memory_format=torch.contiguous_format).float()

    input_ids = tokenizer.pad({"input_ids": input_ids}, padding=True,
                                return_tensors="pt").input_ids
    batch = {
        "input_ids": input_ids,
        "pixel_values": pixel_values,
    }

    return batch

datamoule = UniversalDataModule(
    tokenizer=tokenizer, collate_fn=collate_fn, args=args, datasets=datasets)

lr_monitor = LearningRateMonitor(logging_interval='step')
checkpoint_callback = UniversalCheckpoint(args)

trainer = Trainer.from_argparse_args(args,
                                        callbacks=[
                                            lr_monitor,
                                            checkpoint_callback])

trainer.fit(model, datamoule, ckpt_path=args.load_ckpt_path)

Loading folder data from /content/Fengshenbang-LM/fengshen/examples/finetune_taiyi_stable_diffusion/demo_dataset/part_0.
Done loading data. Len of images: 6


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


--------warning no checkpoint found--------, remove args
Total steps: 6000


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/setup.py:178: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  category=PossibleUserWarning,
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:97: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  category=PossibleUserWarning,
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:106: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW inst

Training: 0it [00:00, ?it/s]